In [1]:
# Importing libraries
import sqlite3
import pandas as pd
import numpy as np
import shutil
import os
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)

In [2]:
# Check content of the ewip_dw database
ewip_dw_conn = sqlite3.connect('ewip_data_warehouse.db')
ewip_dw_tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", ewip_dw_conn)
print("\nEWIP_DW Database Tables:")
print(ewip_dw_tables)

for table in ewip_dw_tables['name']:
    print(f"\nContent of {table} table in ewip_dw database:")
    print(pd.read_sql_query(f"SELECT * FROM {table} LIMIT 10", ewip_dw_conn))

# Establish connection to the DW database
c = ewip_dw_conn.cursor()
ewip_dw_conn.commit()


EWIP_DW Database Tables:
                                name
0                           dim_date
1                         dim_seller
2                           dim_item
3                          dim_staff
4  fact_platform_performance_summary
5       fact_sku_performance_summary
6    fact_seller_performance_summary
7            fact_staff_prod_summary
8             fact_warehouse_summary

Content of dim_date table in ewip_dw database:
   date_key        date  quarter  year  month  day  day_of_week day_type  is_holiday  is_campaign_day  is_salary_day
0  20240101  2024-01-01        1  2024      1    1            1  Weekday           1                0              0
1  20240102  2024-01-02        1  2024      1    2            2  Weekday           0                1              0
2  20240103  2024-01-03        1  2024      1    3            3  Weekday           0                1              0
3  20240104  2024-01-04        1  2024      1    4            4  Weekday           1    

1) PLATFORM PERFORMANCE. What are the monthly historical platform performance and month-on-month (MoM) growth in terms of average daily order (ADO), average daily gross merchandise value (ADGMV), average active buyers, average active shops, and average order-to-delivery (OTD) time?

In [3]:
query = """
WITH monthly_platform_metrics AS (
    SELECT
        strftime('%Y-%m', d.date) AS month,
        SUM(f."1d_ado") AS total_ado,
        SUM(f."1d_adgmv") AS total_adgmv,
        SUM(f."1d_avg_active_buyers") AS total_active_buyers,
        SUM(f."1d_avg_active_shops") AS total_active_shops,
        SUM(f."1d_otd_time") AS total_otd_time,
        strftime('%d', MAX(d.date)) AS days_in_month
    FROM fact_platform_performance_summary f
    JOIN dim_date d ON f.date_key = d.date_key
    GROUP BY strftime('%Y-%m', d.date)
)
SELECT
    month,
    total_ado / days_in_month AS avg_daily_ado,
    total_adgmv / days_in_month AS avg_daily_adgmv,
    total_active_buyers / days_in_month AS avg_daily_active_buyers,
    total_active_shops / days_in_month AS avg_daily_active_shops,
    total_otd_time / days_in_month AS avg_daily_otd_time,
    (total_ado / days_in_month) - LAG(total_ado / days_in_month) OVER (ORDER BY month) AS ado_growth,
    (total_adgmv / days_in_month) - LAG(total_adgmv / days_in_month) OVER (ORDER BY month) AS adgmv_growth,
    (total_active_buyers / days_in_month) - LAG(total_active_buyers / days_in_month) OVER (ORDER BY month) AS active_buyers_growth,
    (total_active_shops / days_in_month) - LAG(total_active_shops / days_in_month) OVER (ORDER BY month) AS active_shops_growth,
    (total_otd_time / days_in_month) - LAG(total_otd_time / days_in_month) OVER (ORDER BY month) AS otd_growth
FROM monthly_platform_metrics
ORDER BY month;
"""

# Execute the query and load the result into a DataFrame
df_metrics = pd.read_sql_query(query, ewip_dw_conn)
print(df_metrics)



      month  avg_daily_ado  avg_daily_adgmv  avg_daily_active_buyers  avg_daily_active_shops  avg_daily_otd_time  ado_growth  adgmv_growth  active_buyers_growth  active_shops_growth  otd_growth
0   2024-01           2664     25396.038683               652.439510              245.066270            2.867120         NaN           NaN                   NaN                  NaN         NaN
1   2024-02           3194     32415.199616               584.974115              256.582252            3.021688       530.0   7019.160933            -67.465395            11.515982    0.154568
2   2024-03           2712     31303.767527               568.337099              239.165396            3.032673      -482.0  -1111.432089            -16.637016           -17.416856    0.010985
3   2024-04           2760     29470.010737               496.887083              289.525208            2.728260        48.0  -1833.756790            -71.450016            50.359812   -0.304413
4   2024-05           2445    

2) ITEMS PERFORMANCE. What are the top item categories in terms of monthly ADO and ADGMV? Which item categories have the highest month-on-month ADO and ADGMV growth?

In [4]:
query = """
WITH monthly_category_metrics AS (
    SELECT
        strftime('%Y-%m', d.date) AS month,
        i.category_lvl_1 AS category,
        SUM(f."1d_ado") AS total_ado,
        SUM(f."1d_adgmv") AS total_adgmv,
        strftime('%d', MAX(d.date)) AS days_in_month
    FROM fact_sku_performance_summary f
    JOIN dim_date d ON f.date_key = d.date_key
    JOIN dim_item i ON f.sku_id = i.sku_id
    GROUP BY strftime('%Y-%m', d.date), i.category_lvl_1
),
category_rankings AS (
    SELECT
        month,
        category,
        total_ado / days_in_month AS avg_daily_ado,
        total_adgmv / days_in_month AS avg_daily_adgmv,
        RANK() OVER (PARTITION BY month ORDER BY total_ado / days_in_month DESC) AS ado_rank,
        RANK() OVER (PARTITION BY month ORDER BY total_adgmv / days_in_month DESC) AS adgmv_rank,
        (total_ado / days_in_month) - LAG(total_ado / days_in_month) OVER monthly_window AS ado_growth,
        (total_adgmv / days_in_month) - LAG(total_adgmv / days_in_month) OVER monthly_window AS adgmv_growth
    FROM monthly_category_metrics
    WINDOW monthly_window AS (PARTITION BY category ORDER BY month)
)
SELECT 
    *,
    RANK() OVER (PARTITION BY month ORDER BY ado_growth DESC) AS ado_growth_rank,
    RANK() OVER (PARTITION BY month ORDER BY adgmv_growth DESC) AS adgmv_growth_rank
FROM category_rankings
WHERE ado_rank <= 10 OR adgmv_rank <= 10
ORDER BY month, ado_rank, adgmv_rank;
"""

# Execute the query and load the result into a DataFrame
df_metrics = pd.read_sql_query(query, ewip_dw_conn)
print(df_metrics)


      month     category  avg_daily_ado  avg_daily_adgmv  ado_rank  adgmv_rank  ado_growth  adgmv_growth  ado_growth_rank  adgmv_growth_rank
0   2024-01     Clothing          35910    712366.726731         1           1         NaN           NaN                1                  1
1   2024-01         Home          32996    639637.944678         2           2         NaN           NaN                1                  1
2   2024-01  Electronics          30588    602581.232542         3           3         NaN           NaN                1                  1
3   2024-01         Toys          29093    580678.947072         4           4         NaN           NaN                1                  1
4   2024-02     Clothing          35792    703818.528909         1           1      -118.0  -8548.197822                3                  4
5   2024-02         Home          32647    636306.782483         2           2      -349.0  -3331.162195                4                  3
6   2024-02  

3) SHOP PERFORMANCE. What are the top-performing shop categories in terms of monthly ADO and ADGMV? What shop categories contribute to more than 10% of the platform ADGMV?


In [5]:
query = """
WITH monthly_shop_category_metrics AS (
    SELECT
        strftime('%Y-%m', d.date) AS month,
        s.shop_category,
        SUM(f."1d_ado") AS total_ado,
        SUM(f."1d_adgmv") AS total_adgmv,
        strftime('%d', MAX(d.date)) AS days_in_month
    FROM fact_seller_performance_summary f
    JOIN dim_date d ON f.date_key = d.date_key
    JOIN dim_seller s ON f.shop_id = s.shop_id
    GROUP BY strftime('%Y-%m', d.date) , s.shop_category
),
shop_category_contributions AS (
    SELECT
        month,
        shop_category,
        total_ado / days_in_month AS avg_daily_ado,
        total_adgmv / days_in_month AS avg_daily_adgmv,
        SUM(total_adgmv) OVER (PARTITION BY month) / days_in_month AS total_platform_adgmv,
        RANK() OVER (PARTITION BY month ORDER BY total_adgmv / days_in_month DESC) AS adgmv_rank
    FROM monthly_shop_category_metrics
)
SELECT
    month,
    shop_category,
    avg_daily_ado,
    avg_daily_adgmv,
    (avg_daily_adgmv / total_platform_adgmv) * 100 AS adgmv_percentage
FROM shop_category_contributions
WHERE adgmv_rank <= 10 OR (avg_daily_adgmv / total_platform_adgmv) > 0.1
ORDER BY month, adgmv_rank;
"""

# Execute the query and load the result into a DataFrame
df_metrics = pd.read_sql_query(query, ewip_dw_conn)
print(df_metrics)


      month shop_category  avg_daily_ado  avg_daily_adgmv  adgmv_percentage
0   2024-01      Clothing           7573    152936.563918         30.472885
1   2024-01   Electronics           7672    146050.419127         29.100808
2   2024-01          Toys           5851    120077.437640         23.925645
3   2024-01          Home           4454     82813.119319         16.500663
4   2024-02      Clothing           7693    156185.882020         31.016859
5   2024-02   Electronics           7585    149814.895324         29.751649
6   2024-02          Toys           5770    115536.022145         22.944228
7   2024-02          Home           4414     82014.776838         16.287264
8   2024-03   Electronics           7583    153790.473229         30.196296
9   2024-03      Clothing           7605    151756.930371         29.797016
10  2024-03          Toys           5881    117143.075543         23.000690
11  2024-03          Home           4109     86611.966860         17.005999
12  2024-04 

4) WAREHOUSE PERFORMANCE. What is the monthly historical performance and MoM growth of the overall warehouse and each warehouse in terms of ADO, average daily item (ADI) count, productivity rate, and idle rate?

In [6]:
query = """
WITH monthly_warehouse_metrics AS (
    SELECT
        strftime('%Y-%m', d.date) AS month,
        w.wh_key,
        SUM(w."1d_ado") AS total_ado,
        SUM(w."1d_adi") AS total_adi,
        SUM(w."1d_prod_rate") AS total_prod_rate,
        SUM(w."1d_idle_rate") AS total_idle_rate,
        strftime('%d', MAX(d.date)) AS days_in_month
    FROM fact_warehouse_summary w
    JOIN dim_date d ON w.date_key = d.date_key
    GROUP BY strftime('%Y-%m', d.date), w.wh_key
),
overall_warehouse_metrics AS (
    SELECT
        month,
        'Overall' AS wh_key,
        SUM(total_ado) AS total_ado,
        SUM(total_adi) AS total_adi,
        SUM(total_prod_rate) AS total_prod_rate,
        SUM(total_idle_rate) AS total_idle_rate,
        MAX(days_in_month) AS days_in_month
    FROM monthly_warehouse_metrics
    GROUP BY month
)
SELECT
    wh_key,
    month,
    total_ado / days_in_month AS avg_daily_ado,
    total_adi / days_in_month AS avg_daily_adi,
    total_prod_rate / days_in_month AS avg_daily_prod_rate,
    total_idle_rate / days_in_month AS avg_daily_idle_rate,
    (total_ado / days_in_month) - LAG(total_ado / days_in_month) OVER (PARTITION BY wh_key ORDER BY month) AS ado_growth,
    (total_adi / days_in_month) - LAG(total_adi / days_in_month) OVER (PARTITION BY wh_key ORDER BY month) AS adi_growth,
    (total_prod_rate / days_in_month) - LAG(total_prod_rate / days_in_month) OVER (PARTITION BY wh_key ORDER BY month) AS prod_rate_growth,
    (total_idle_rate / days_in_month) - LAG(total_idle_rate / days_in_month) OVER (PARTITION BY wh_key ORDER BY month) AS idle_rate_growth
FROM (
    SELECT * FROM monthly_warehouse_metrics
    UNION ALL
    SELECT * FROM overall_warehouse_metrics
) combined_metrics
ORDER BY month, wh_key;
"""

# Execute the query and load the result into a DataFrame
df_metrics = pd.read_sql_query(query, ewip_dw_conn)
print(df_metrics)


      wh_key    month  avg_daily_ado  avg_daily_adi  avg_daily_prod_rate  avg_daily_idle_rate  ado_growth  adi_growth  prod_rate_growth  idle_rate_growth
0          1  2024-01            512            247             0.983228             0.298382         NaN         NaN               NaN               NaN
1          2  2024-01            528            262             1.012711             0.295851         NaN         NaN               NaN               NaN
2          3  2024-01            590            246             0.962967             0.310055         NaN         NaN               NaN               NaN
3          4  2024-01            536            246             0.984974             0.301745         NaN         NaN               NaN               NaN
4          5  2024-01            512            244             1.029501             0.288105         NaN         NaN               NaN               NaN
5          6  2024-01            471            265             1.009494    

5) STAFF PERFORMANCE. Who are the warehouse staff who have last 30-day productivity less than 90% of the average rate of the top 10 staff?

In [7]:
query = """
WITH staff_productivity AS (
    SELECT
        s.staff_key,
        s.staff_email,
        SUM(f.prod_rate) AS total_prod_rate,
        30.000 AS days_in_month,
        SUM(f.prod_rate) / 30.000 AS avg_prod_rate
    FROM fact_staff_prod_summary f
    JOIN dim_date d ON f.date_key = d.date_key
    JOIN dim_staff s ON f.staff_key = s.staff_key
    WHERE d.date >= DATE('now', '-30 days')
    GROUP BY s.staff_key
),
top_10_avg_productivity AS (
    SELECT
        AVG(avg_prod_rate) AS avg_top_10_productivity
    FROM (
        SELECT
            avg_prod_rate
        FROM staff_productivity
        ORDER BY avg_prod_rate DESC
        LIMIT 10
    ) top_10
)
SELECT
    sp.staff_key,
    sp.staff_email,
    sp.avg_prod_rate,
    t10.avg_top_10_productivity
FROM staff_productivity sp, top_10_avg_productivity t10
WHERE sp.avg_prod_rate < 0.9 * t10.avg_top_10_productivity
ORDER BY sp.avg_prod_rate;
"""

# Execute the query and load the result into a DataFrame
df_metrics = pd.read_sql_query(query, ewip_dw_conn)
print(df_metrics)


   staff_key                   staff_email  avg_prod_rate  avg_top_10_productivity
0         28           mpotter@example.net       1.059016                 1.232146
1         23         uguerrero@example.org       1.060031                 1.232146
2         22      joshuabarnes@example.org       1.093157                 1.232146
3         46  phillipsveronica@example.net       1.102640                 1.232146
4         44       gloverdiana@example.net       1.104230                 1.232146
